In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import time
import gc

In [16]:
gc.collect()
torch.cuda.empty_cache()

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
DATASET_PATH = r"C:\Users\krish\Downloads\oxford flowers 102.v1i.folder"  #https://universe.roboflow.com/ck6jnertibvrhdvxefrfsu2pvcz2/oxford-flowers-102/dataset/1

In [5]:
'''transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Keep at 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])'''
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),  # Flip images randomly
    transforms.RandomRotation(30),           # Rotate images
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # Adjust colors
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1)),  # Small shifts
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [6]:
train_set = datasets.ImageFolder(root=f"{DATASET_PATH}/train", transform=transform)
val_set = datasets.ImageFolder(root=f"{DATASET_PATH}/valid", transform=transform)
test_set = datasets.ImageFolder(root=f"{DATASET_PATH}/test", transform=transform)

In [7]:
batch_size = 32  
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

In [8]:
class InceptionBlock(nn.Module):
    def __init__(self, in_channels, out_1x1, red_3x3, out_3x3, red_5x5, out_5x5, pool_proj):
        super(InceptionBlock, self).__init__()
        # Branch 1: 1x1 convolution
        self.branch1 = nn.Sequential(
            nn.Conv2d(in_channels, out_1x1, kernel_size=1),
            nn.BatchNorm2d(out_1x1),
            nn.ReLU(inplace=True)
        )
        # Branch 2: 1x1 convolution followed by 3x3 convolution
        self.branch2 = nn.Sequential(
            nn.Conv2d(in_channels, red_3x3, kernel_size=1),
            nn.BatchNorm2d(red_3x3),
            nn.ReLU(inplace=True),
            nn.Conv2d(red_3x3, out_3x3, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_3x3),
            nn.ReLU(inplace=True)
        )
        # Branch 3: 1x1 convolution followed by 5x5 convolution
        self.branch3 = nn.Sequential(
            nn.Conv2d(in_channels, red_5x5, kernel_size=1),
            nn.BatchNorm2d(red_5x5),
            nn.ReLU(inplace=True),
            nn.Conv2d(red_5x5, out_5x5, kernel_size=5, padding=2),
            nn.BatchNorm2d(out_5x5),
            nn.ReLU(inplace=True)
        )
        # Branch 4: 3x3 max pooling followed by 1x1 convolution
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels, pool_proj, kernel_size=1),
            nn.BatchNorm2d(pool_proj),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)
        # Concatenate the outputs along the channel dimension
        return torch.cat([branch1, branch2, branch3, branch4], 1)

In [9]:
class MiniGoogLeNet(nn.Module):
    def __init__(self, num_classes=102):
        super(MiniGoogLeNet, self).__init__()
        # Initial convolution and pooling (similar to GoogLeNet's stem)
        self.pre_layers = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),  # 3 -> 64 channels
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        # Inception 3a: input channels = 64
        self.inception3a = InceptionBlock(in_channels=64,
                                          out_1x1=32,
                                          red_3x3=48, out_3x3=64,
                                          red_5x5=8,  out_5x5=16,
                                          pool_proj=16)
        # Output channels from inception3a: 32 + 64 + 16 + 16 = 128

        # Inception 3b: input channels = 128
        self.inception3b = InceptionBlock(in_channels=128,
                                          out_1x1=64,
                                          red_3x3=64, out_3x3=96,
                                          red_5x5=16, out_5x5=48,
                                          pool_proj=32)
        # Output channels from inception3b: 64 + 96 + 48 + 32 = 240

        # A max pooling layer to reduce spatial dimensions
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Inception 4a: input channels = 240
        self.inception4a = InceptionBlock(in_channels=240,
                                          out_1x1=128,
                                          red_3x3=128, out_3x3=192,
                                          red_5x5=32,  out_5x5=96,
                                          pool_proj=64)
        # Output channels from inception4a: 128 + 192 + 96 + 64 = 480

        # Global average pooling, dropout and final classifier
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(0.4)
        self.fc = nn.Linear(480, num_classes)

    def forward(self, x):
        x = self.pre_layers(x)
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool(x)
        x = self.inception4a(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)
        return x

# Instantiate and move the model to device
model = MiniGoogLeNet(num_classes=102).to(device)

In [10]:
model = MiniGoogLeNet(num_classes=102).to(device)

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [12]:
num_epochs = 50
best_val_acc = 0.0

In [13]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    start_time = time.time()

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

    train_acc = 100 * correct / total
    train_loss = running_loss / len(train_loader)

    # **Validation**
    model.eval()
    correct = 0
    total = 0
    val_loss = 0.0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

    val_acc = 100 * correct / total
    val_loss = val_loss / len(val_loader)

    # **Save Best Model**
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")

    end_time = time.time()
    print(f"Epoch [{epoch+1}/{num_epochs}] - "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% - "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}% - "
          f"Time: {end_time - start_time:.2f}s")

print("Training complete. Best Validation Accuracy:", best_val_acc)

Epoch [1/50] - Train Loss: 3.9386, Train Acc: 11.73% - Val Loss: 3.5057, Val Acc: 15.26% - Time: 34.69s
Epoch [2/50] - Train Loss: 3.2524, Train Acc: 21.48% - Val Loss: 3.1131, Val Acc: 24.40% - Time: 34.18s
Epoch [3/50] - Train Loss: 2.9400, Train Acc: 26.74% - Val Loss: 2.8300, Val Acc: 28.75% - Time: 33.78s
Epoch [4/50] - Train Loss: 2.7125, Train Acc: 31.37% - Val Loss: 2.8161, Val Acc: 28.90% - Time: 33.81s
Epoch [5/50] - Train Loss: 2.5868, Train Acc: 34.54% - Val Loss: 2.5989, Val Acc: 31.54% - Time: 33.78s
Epoch [6/50] - Train Loss: 2.4509, Train Acc: 37.23% - Val Loss: 2.4213, Val Acc: 36.33% - Time: 34.00s
Epoch [7/50] - Train Loss: 2.3000, Train Acc: 41.39% - Val Loss: 2.3301, Val Acc: 40.00% - Time: 34.23s
Epoch [8/50] - Train Loss: 2.2055, Train Acc: 41.93% - Val Loss: 2.2304, Val Acc: 42.44% - Time: 34.13s
Epoch [9/50] - Train Loss: 2.1150, Train Acc: 44.86% - Val Loss: 2.2105, Val Acc: 43.91% - Time: 34.19s
Epoch [10/50] - Train Loss: 2.0363, Train Acc: 45.95% - Val Loss

In [14]:
# **Testing**
model.load_state_dict(torch.load("best_model.pth"))
model.eval()

correct = 0
total = 0

In [15]:
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

test_acc = 100 * correct / total
print(f"Test Accuracy: {test_acc:.2f}%")

Test Accuracy: 69.59%
